In [1]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt # for making figures
%matplotlib inline

In [2]:
# read in all the words
words = open('names.txt', 'r').read().splitlines()
words[:8]

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [3]:
# build the vocabulary of characters and mappings to/from integers
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
print(f'stoi: {stoi}')
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
vocab_size = len(itos)
print(f'itos: {itos}')
print(f'Size: {vocab_size}')

stoi: {'a': 1, 'b': 2, 'c': 3, 'd': 4, 'e': 5, 'f': 6, 'g': 7, 'h': 8, 'i': 9, 'j': 10, 'k': 11, 'l': 12, 'm': 13, 'n': 14, 'o': 15, 'p': 16, 'q': 17, 'r': 18, 's': 19, 't': 20, 'u': 21, 'v': 22, 'w': 23, 'x': 24, 'y': 25, 'z': 26}
itos: {1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}
Size: 27


In [4]:
# build the dataset
block_size = 3 # context length: how many characters do we take to predict the next one?

def build_dataset(words):  
  X, Y = [], []
  for w in words:

    #print(w)
    context = [0] * block_size
    for ch in w + '.':
      ix = stoi[ch]
      X.append(context)
      Y.append(ix)
      #print(''.join(itos[i] for i in context), '--->', itos[ix])
      context = context[1:] + [ix] # crop and append

  X = torch.tensor(X)
  Y = torch.tensor(Y)
  print(X.shape, Y.shape)
  return X, Y

import random
random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

Xtr, Ytr = build_dataset(words[:n1])
Xdev, Ydev = build_dataset(words[n1:n2])
Xte, Yte = build_dataset(words[n2:])

torch.Size([182625, 3]) torch.Size([182625])
torch.Size([22655, 3]) torch.Size([22655])
torch.Size([22866, 3]) torch.Size([22866])


In [5]:
# boilerplate code is finished

In [6]:
# utility function to compare gradient calculated manually with gradient from pytorch
def cmp(s, dt, t):
    ex = torch.all(dt == t.grad).item()
    app = torch.allclose(dt, t.grad)
    maxdiff = (dt - t.grad).abs().max().item()
    print(f'{s:15s} | exact {str(ex):5s} | approximate: {str(app):5s} | maxdiff: {maxdiff}')

In [7]:
n_embd = 10 # the dimensionality of the character embedding vectors
n_hidden = 64 # the number of neurons in the hidden layer of the MLP

g = torch.Generator().manual_seed(2147483647) # for reproducibility
C  = torch.randn((vocab_size, n_embd),            generator=g)
# Layer 1
W1 = torch.randn((n_embd * block_size, n_hidden), generator=g) * (5/3)/((n_embd * block_size)**0.5)
b1 = torch.randn(n_hidden,                        generator=g) * 0.1 # using b1 just for fun, it's useless because of BN
# Layer 2
W2 = torch.randn((n_hidden, vocab_size),          generator=g) * 0.1
b2 = torch.randn(vocab_size,                      generator=g) * 0.1
# BatchNorm parameters
bngain = torch.randn(1, n_hidden)*0.1 + 1.0
bnbias = torch.randn(1, n_hidden)*0.1

# Note: I am initializating many of these parameters in non-standard ways
# because sometimes initializating with e.g. all zeros could mask an incorrect
# implementation of the backward pass.

parameters = [C, W1, b1, W2, b2, bngain, bnbias]
print(sum(p.nelement() for p in parameters)) # number of parameters in total
for p in parameters:
  p.requires_grad = True

4137


In [8]:
# mini-batch
batch_size = 32
n = batch_size #shorter variable for convernience

ix = torch.randint(0, Xtr.shape[0], (batch_size, ), generator=g)
Xb, Yb = Xtr[ix], Ytr[ix]

In [9]:
# forward pass
emb = C[Xb]      # embed the characters into vectors *dimension 32x3x10
embcat = emb.view(Xb.shape[0], -1)  # concatenate the vectors   * dimension 32x30

#linear layer
hprebn = embcat@W1 + b1#  hidden layer preactivation

# Batch Normalization Layer
# -----------------------------------------------
bnmeani = 1/n*hprebn.sum(0, keepdim=True)
bndiff = hprebn - bnmeani
bndiff2 = bndiff**2
bnvar = 1/(n-1)*(bndiff2).sum(0, keepdim=True) # note: Bessel's correction (dividing by n-1, not n)
bnvar_inv = (bnvar + 1e-5)**-0.5
bnraw = bndiff * bnvar_inv
hpreact = bngain * bnraw + bnbias

# -----------------------------------------------

# non-linearity using tanh

h = torch.tanh(hpreact) # hidden layer

# linear layer 2
logits = h @ W2 + b2    # output layer

# cross entropy
# loss = F.cross_entropy(logits, Yb)   # loss function
logit_maxes = logits.max(1, keepdim=True).values
norm_logits = logits - logit_maxes # subtract max for numerical stability
counts = norm_logits.exp()
counts_sum = counts.sum(1, keepdims=True)
counts_sum_inv = counts_sum**-1 # if I use (1.0 / counts_sum) instead then I can't get backprop to be bit exact...
probs = counts * counts_sum_inv
logprobs = probs.log()
loss = -logprobs[range(n), Yb].mean()


#backward pass
for p in parameters:
    p.grad = None
for t in [logprobs, probs, counts, counts_sum, counts_sum_inv, # afaik there is no cleaner way
          norm_logits, logit_maxes, logits, h, hpreact, bnraw,
         bnvar_inv, bnvar, bndiff2, bndiff, hprebn, bnmeani,
         embcat, emb]:
  t.retain_grad()
loss.backward()
loss

tensor(3.3389, grad_fn=<NegBackward0>)

In [36]:
## dlogprobs i.e. dloss/dlogprobs
# cmp('logprobs', dlogprobs, logprobs)
# dlogprobs = ???

#loss = -logprobs[range(n), Yb].mean()

# logprobs = probs.log()

logprobs.shape #32 x 27

# mean is same as taking average so loss = -logprobs[range(n), Yb].sum()/n

# smaller example loss = -(a+b+c)/3 => 
# dloss/da = -1/3, -1/3, -1/3

# dloss/dlogprobs = -1/n
dlogprobs = torch.zeros_like(logprobs)
dlogprobs[range(n), Yb] = -1/n

cmp("dlogprobs", dlogprobs, logprobs)

## cmp('probs', dprobs, probs)
# logprobs = probs.log()
# dprobs = ???
# dloss/dprobs = dllogprobs/dlog * dloss/dlogprobs

# y = log(x) => dy/dx = 1/x
# 1/probs is local derivative and then dlogprobs get chained 
# if prob assigned is low, it gets higher derivative as it needs bigger change
dprobs = (1/probs) * dlogprobs
cmp('probs', dprobs, probs)

# cmp('counts_sum_inv', dcounts_sum_inv, counts_sum_inv)
# probs = counts * counts_sum_inv
# dcounts_sum_inv = ??
# dloss/dcounts_sum_inv = dprobs/dcounts_sum_inv * dloss/dprobs


# y = a * x => dy/dx = a
# local derivative = > counts

# counts * counts_sum_inv  --> two openration happens here, multiplication and replication(broadcasting) 
# a [3x3] b[3x1]
# a11*b1 + a12*b1+ a13*b1
# a21*b2 + a22*b2 + a23*b2
# a31*b3 + a32*b2 + a33*b3 

# print(probs.shape, counts.shape, counts_sum_inv.shape)



dcounts_sum_inv = (counts * dprobs).sum(1, keepdim=True)  # sum is from backpropagation through broadcasting/replication of columns
cmp('counts_sum_inv', dcounts_sum_inv, counts_sum_inv)

# cmp('counts_sum', dcounts_sum, counts_sum)
# counts_sum = counts.sum(1, keepdims=True)
# counts_sum_inv = counts_sum**-1
# probs = counts * counts_sum_inv

# dcounts_sum = ?

# dloss/dcounts_sum = dcounts_sum_inv/dcounts_sum * dloss/dcounts_sum_inv

# y = a*x
dcounts = counts_sum_inv * dprobs
dcounts_sum = (-1 * counts_sum**-2) * dcounts_sum_inv

cmp('counts_sum', dcounts_sum, counts_sum)

# cmp('counts', dcounts, counts)
# counts_sum = counts.sum(1, keepdims=True)

# dcounts is used two times
# dcounts = ?
# dcounts/dloss = dcounts_sum/dcounts * dloss/dcounts_sum

# retrace through sum for differentiation

dcounts += torch.ones_like(counts) * dcounts_sum
cmp('counts', dcounts, counts)

# cmp('norm_logits', dnorm_logits, norm_logits)
# counts = norm_logits.exp()
# dnorm_logits = ?

# dloss/dnorm_logits = dcounts/ dnorm_logits * dloss/dcounts

dnorm_logits = norm_logits.exp() * dcounts
cmp('norm_logits', dnorm_logits, norm_logits)

# cmp('logit_maxes', dlogit_maxes, logit_maxes)
  
# logit_maxes = logits.max(1, keepdim=True).values
# norm_logits = logits - logit_maxes # subtract max for numerical stability


# dlogit_maxes = ?

# dloss/dlogit_maxes = dnorm_logits / dlogit_maxes * dloss/dnorm_logits

# local derivative ?

# c11, c12, c13 = a11, a12, a13 - b1
# c21, c22, c23 = a21, a22, a23 - b2
# c31, c32, c33 = a31, a32, a33 - b3
# so eg c22 = a22 - b2

# print(norm_logits.shape, logits.shape , logit_maxes.shape)
dlogits = dnorm_logits.clone() # dlogits = dnorm_logits/dlogits * dloss/dnorm_logits => dnorm_logits/dlogits = 1

dlogit_maxes = (-dnorm_logits).sum(1, keepdim=True)   # replication/broadcasting in subtraction
cmp('logit_maxes', dlogit_maxes, logit_maxes)


# cmp('logits', dlogits, logits)
# logit_maxes = logits.max(1, keepdim=True).values

# dlogits =?
# dloss/dlogits = dlogit_maxes/dlogit * dloss/dlogit_maxes

# a11, a12, a13 -> a12
# b21, b22, b23 -> b23
# c31, c32, c33 -> c31

temp_logits = torch.zeros_like(logits)
temp_logits[range(32), logits.argmax(dim=1)] = 1
dlogits += temp_logits * dlogit_maxes


cmp('logits', dlogits, logits)

# logits = h @ W2 + b2  
# cmp('h', dh, h)

# dloss/dh = dlogits/dh * dloss/dlogits

# print(logits.shape, h.shape, W2.shape, b2.shape, dlogits.shape)
# TODO: check solution by manually solving matrix multiplication differentiation

dh = (dlogits @ W2.transpose(0, 1))
cmp('h', dh, h)


# logits = h @ W2 + b2  
# cmp('W2', dW2, W2)

# dW2=?
# dloss/dW2 = dlogits/dW2 * dloss/dlogits
# print(logits.shape, h.shape, W2.shape, dlogits.shape)
dW2 = (h.transpose(0,1)@dlogits)
cmp('W2', dW2, W2)

# cmp('b2', db2, b2)
# logits = h @ W2 + b2 

# db2=?
# dloss/db2 = dlogits/db2 * dloss/dlogits
# print(logits.shape, h.shape, W2.shape, b2a.shape, dlogits.shape)
db2 = dlogits.sum(0, keepdim=False)
cmp('b2', db2, b2)

# cmp('hpreact', dhpreact, hpreact)
# h = torch.tanh(hpreact) 
# dhpreact = ?
# dloss/dhpreact = dh/dhpreacht * dloss/dh

dhpreact = (1-h**2) * dh
cmp('hpreact', dhpreact, hpreact)

dlogprobs       | exact True  | approximate: True  | maxdiff: 0.0
probs           | exact True  | approximate: True  | maxdiff: 0.0
counts_sum_inv  | exact True  | approximate: True  | maxdiff: 0.0
counts_sum      | exact True  | approximate: True  | maxdiff: 0.0
counts          | exact True  | approximate: True  | maxdiff: 0.0
norm_logits     | exact True  | approximate: True  | maxdiff: 0.0
logit_maxes     | exact True  | approximate: True  | maxdiff: 0.0
logits          | exact True  | approximate: True  | maxdiff: 0.0
h               | exact True  | approximate: True  | maxdiff: 0.0
W2              | exact True  | approximate: True  | maxdiff: 0.0
b2              | exact True  | approximate: True  | maxdiff: 0.0
hpreact         | exact True  | approximate: True  | maxdiff: 0.0


In [44]:
# cmp('bngain', dbngain, bngain)
# hpreact = bngain * bnraw + bnbias

# dbngain = ?
# dloss/dbngain = dhpreact/dbngain * dloss/dhpreact
# print(hpreact.shape, bngain.shape, bnraw.shape, bnbias.shape)
# torch.Size([32, 64]) torch.Size([1, 64]) torch.Size([32, 64]) torch.Size([1, 64])
dbngain = (bnraw * dhpreact).sum(0, keepdim=True)

cmp('bngain', dbngain, bngain)

bngain          | exact True  | approximate: True  | maxdiff: 0.0


In [43]:
# cmp('bnbias', dbnbias, bnbias)
# hpreact = bngain * bnraw + bnbias

# dbnbias = ?
# dloss/dbnbias = dhpreact/dbnbias * dloss/dhpreact
dbnbias = (dhpreact).sum(0, keepdim=True)
cmp('bnbias', dbnbias, bnbias)

bnbias          | exact True  | approximate: True  | maxdiff: 0.0


In [48]:
# cmp('bnraw', dbnraw, bnraw)
# hpreact = bngain * bnraw + bnbias

# dbnraw=?
# dloss/dbnraw = dhpreact/dbnraw * dloss/dhpreact
dbnraw = dhpreact
dbnraw.shape
# cmp('bnraw', dbnraw, bnraw)

torch.Size([32, 64])

In [20]:
# Exercise 1: backprop through the whole thing manually,
# backpropagating through exactly all of the variables
# as they are defined in the forward pass above, one by one

# -----------------
# YOUR CODE HERE :)
# -----------------

dlogprobs = torch.zeros_like(logprobs)                 # dloss/dlogprobs
dlogprobs[range(n), Yb] = -1.0/n                

dprobs = (1.0/probs )* dlogprobs    #  dloss/dprobs = dlogprobs/dprobs * dloss/dlogprobs

dcounts_sum_inv = (counts * dprobs).sum(1, keepdim=True)      # dloss/dcounts_sum_inv = dprobs/dcounts_sum_inv * dloss/dprobs
# have to sum because there are two operations.. multiplication and transformation 

cmp('logprobs', dlogprobs, logprobs)
cmp('probs', dprobs, probs)
cmp('counts_sum_inv', dcounts_sum_inv, counts_sum_inv)
# cmp('counts_sum', dcounts_sum, counts_sum)
# cmp('counts', dcounts, counts)
# cmp('norm_logits', dnorm_logits, norm_logits)
# cmp('logit_maxes', dlogit_maxes, logit_maxes)
# cmp('logits', dlogits, logits)
# cmp('h', dh, h)
# cmp('W2', dW2, W2)
# cmp('b2', db2, b2)
# cmp('hpreact', dhpreact, hpreact)
# cmp('bngain', dbngain, bngain)
# cmp('bnbias', dbnbias, bnbias)
# cmp('bnraw', dbnraw, bnraw)
# cmp('bnvar_inv', dbnvar_inv, bnvar_inv)
# cmp('bnvar', dbnvar, bnvar)
# cmp('bndiff2', dbndiff2, bndiff2)
# cmp('bndiff', dbndiff, bndiff)
# cmp('bnmeani', dbnmeani, bnmeani)
# cmp('hprebn', dhprebn, hprebn)
# cmp('embcat', dembcat, embcat)
# cmp('W1', dW1, W1)
# cmp('b1', db1, b1)
# cmp('emb', demb, emb)
# cmp('C', dC, C)

logprobs        | exact True  | approximate: True  | maxdiff: 0.0
probs           | exact True  | approximate: True  | maxdiff: 0.0
counts_sum_inv  | exact True  | approximate: True  | maxdiff: 0.0
